In [65]:
import torch
from task1.test1 import define_model as model1
import numpy as np

In [61]:
cfg1 = {
        "vocab_size": 180+1,
        "device": "cuda:0",
        "block_size": 122 //2,
        "n_embd": 64,
        "n_head": 4,
        "n_layer": 2,
        "dropout": 0.1,
        "n_hidden": 64,
        "use_agent_mask": True,
        'model_save_path': "weights/model_task1.pth",
        'ta_sliding_window':1,
        'use_model':'sd',
        'use_ne':True
        }
cfg2 = {
    'device':'cuda',
    'block_size':122 //2, # max length of trajectory
    'n_embd':10,
    'n_head':1,
    'n_layer':1,
    'dropout':0.1,
    'n_hidden':10,
    'n_embed_adj':10,
    'vocab_size':100+1,
    'window_size':2,
    'max_iters':10,
    'learning_rate':0.001,
    'batch_size':1,
    'model_save_path': "weights/model_task2.pth"
    }

cfg3 = {
    'vocab_size':101,
    'n_embd' : 64,
    'n_head' : 4,
    'n_layer' : 2,
    'dropout' : 0.1,
    'device' :"cuda",
    "block_size":122 //2,
    'weight_quantization_scale': 20,
    'use_adj_table':True,
    'learning_rate':0.001,
    'max_epochs':100,
    'observe_ratio':0.5,
    'special_mask_value':0.0001,
    'model_save_path': "weights/model_task3.pth"
}

In [55]:
task1_model_path = 'weights/model_task1.pth'

In [62]:
task1_model = model1(cfg1)
task1_model.load_state_dict(torch.load(task1_model_path))
task1_model.eval()

Model: self.time_weight/self.time_weight.mean()=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.])


SpatialTemporalMultiAgentModel(
  (encoder): EncoderHead(
    (token_embedding_table): NormalizedEmbedding(
      (embedding): Embedding(181, 64)
    )
    (position_embedding_table): Embedding(61, 64)
    (in_proj): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (2): SiLU()
    )
    (condition_proj): Sequential(
      (0): Linear(in_features=192, out_features=64, bias=True)
      (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (2): SiLU()
      (3): Linear(in_features=64, out_features=3, bias=True)
    )
  )
  (blocks): ModuleList(
    (0-1): 2 x SpatialTemporalBlock(
      (ta): MultiHeadAttention(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=False)
        )
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=128, bias=True)
      

In [63]:
o,d = np.random.choice(np.arange(1,181),2)
print('Origin:',o,'Destination:',d)

Origin: 48 Destination: 170


In [64]:
traj = task1_model.generate_traj(o,d)

AttributeError: 'SpatialTemporalMultiAgentModel' object has no attribute 'generate_traj'

In [45]:

top_10_values,top_10_indices = torch.topk(task1_output[0,0,0,:],10)

In [46]:
print(top_10_values,top_10_indices)

tensor([36.9776, 22.1486, 18.6211, 17.4208, 16.1031, 15.0057, 14.9232, 13.8469,
        13.8040, 12.8046], device='cuda:0', grad_fn=<TopkBackward0>) tensor([ 45,  29,  43,  52, 169,  28,  32,  48,  30, 102], device='cuda:0')


In [42]:
x['traj'][0,1,0] = top_10_indices[0]
x['reagent_mask'][0,1,0] = 0
task1_output,loss = task1_model(x)
top_10_values,top_10_indices = torch.topk(task1_output[0,1,0,:],10)
print(top_10_values,top_10_indices)

tensor([44.2270, 28.6142, 24.9944, 24.5659, 17.8215, 17.5678, 16.2660, 15.4175,
        15.3664, 14.9853], device='cuda:0', grad_fn=<TopkBackward0>) tensor([ 39,  42, 106,  41, 116,  60,   2,  58,  37,  23], device='cuda:0')
